In [1]:
import sys
import os
import math
import random
import array
from datetime import datetime

import numpy as np
import pandas as pd 

from scipy.stats import lognorm
from scipy.stats import beta
from scipy.stats import binom
from scipy.stats import norm
from scipy.stats import multivariate_normal
from scipy.stats import expon
from scipy.stats import pareto
import scipy.sparse as sp
import scipy.linalg as la
from scipy.stats import dirichlet
 

import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

from itertools import combinations
from itertools import combinations_with_replacement
from itertools import permutations
from itertools import product


from tqdm import trange, notebook

pd.set_option('display.max_columns', None)


In [2]:
os.chdir('D:/Dropbox/Dropbox/uchicago_fourth/uncertaintyInequality')
os.getcwd()

'D:\\Dropbox\\Dropbox\\uchicago_fourth\\uncertaintyInequality'

In [3]:
import gc 
gc.collect()

0

## Creating a wide Panel data

In [24]:
HH_Income_dir = 'pyramid/HHIncome/'
zipFiles = os.listdir(HH_Income_dir)
zipFiles

['household_income_20140131_MS_rev_csv.zip',
 'household_income_20140228_MS_rev_csv.zip',
 'household_income_20140331_MS_rev_csv.zip',
 'household_income_20140430_MS_rev_csv.zip',
 'household_income_20140531_MS_rev_csv.zip',
 'household_income_20140630_MS_rev_csv.zip',
 'household_income_20140731_MS_rev_csv.zip',
 'household_income_20140831_MS_rev_csv.zip',
 'household_income_20140930_MS_rev_csv.zip',
 'household_income_20141031_MS_rev_csv.zip',
 'household_income_20141130_MS_rev_csv.zip',
 'household_income_20141231_MS_rev_csv.zip',
 'household_income_20150131_MS_rev_csv.zip',
 'household_income_20150228_MS_rev_csv.zip',
 'household_income_20150331_MS_rev_csv.zip',
 'household_income_20150430_MS_rev_csv.zip',
 'household_income_20150531_MS_rev_csv.zip',
 'household_income_20150630_MS_rev_csv.zip',
 'household_income_20150731_MS_rev_csv.zip',
 'household_income_20150831_MS_rev_csv.zip',
 'household_income_20150930_MS_rev_csv.zip',
 'household_income_20151031_MS_rev_csv.zip',
 'househol

In [25]:
initalFile = 12
fileName = HH_Income_dir+ zipFiles[initalFile]

df = pd.read_csv(fileName, compression='zip')
dropcols = ['STATE','HR', 'DISTRICT','REGION_TYPE','STRATUM','PSU_ID',
            'MONTH_SLOT','MONTH','RESPONSE_STATUS','HH_NON_RESPONSE_MS',
            'HH_NON_RESPONSE_FOR_COUNTRY_MS','HH_NON_RESPONSE_FOR_STATE_MS',
            'AGE_GROUP','OCCUPATION_GROUP','EDUCATION_GROUP','GENDER_GROUP','SIZE_GROUP']
dropCol = [c for c in df.columns if c.find('WEIGHT')>1] + dropcols
df = df.drop(dropCol,axis=1)

if 'FAMILY_SHIFTED' not in df.columns:
    df['FAMILY_SHIFTED'] = 'N'

keep_same = df.columns[0]
suffix = "_"+fileName[fileName.find('_20')+1:fileName.find('_MS')]
df.columns = ['{}{}'.format(c, '' if c in keep_same else suffix) for c in df.columns]

for f in notebook.tqdm(zipFiles[initalFile+1:71], desc='File', leave=True):
    fileName = HH_Income_dir+ f
    df_toMerge = pd.read_csv(fileName, compression='zip')
    df_toMerge = df_toMerge.drop(dropCol,axis=1)
    if 'FAMILY_SHIFTED' not in df.columns:
        df_toMerge['FAMILY_SHIFTED'] = 'N'

    keep_same = df_toMerge.columns[0]
    suffix = "_"+ f[f.find('_20')+1:f.find('_MS')]
    df_toMerge.columns = ['{}{}'.format(c, '' if c in keep_same else suffix) for c in df_toMerge.columns]

    df = df.merge(df_toMerge,on=['HH_ID'],how='outer',validate="1:1")

    

File:   0%|          | 0/58 [00:00<?, ?it/s]

# Finishing up! and savign the HH panel

In [27]:
df.to_feather('proc_data\HH_Income_Panel.feather')


In [ ]:
list_of_columns = [c for c in df.columns if c.find('REASON_FOR_NON_RESPONSE')>-1]

df['nonResponds'] = 0
for c in list_of_columns:
    df['nonResponds'] = df['nonResponds'] + (df[c]!='No Failure')
     
df['nonResponds'].hist(bins=50)

# Creating a long panel data

In [4]:
HH_Income_dir = 'pyramid/HHIncome/'
zipFiles = os.listdir(HH_Income_dir)
zipFiles

['household_income_20140131_MS_rev_csv.zip',
 'household_income_20140228_MS_rev_csv.zip',
 'household_income_20140331_MS_rev_csv.zip',
 'household_income_20140430_MS_rev_csv.zip',
 'household_income_20140531_MS_rev_csv.zip',
 'household_income_20140630_MS_rev_csv.zip',
 'household_income_20140731_MS_rev_csv.zip',
 'household_income_20140831_MS_rev_csv.zip',
 'household_income_20140930_MS_rev_csv.zip',
 'household_income_20141031_MS_rev_csv.zip',
 'household_income_20141130_MS_rev_csv.zip',
 'household_income_20141231_MS_rev_csv.zip',
 'household_income_20150131_MS_rev_csv.zip',
 'household_income_20150228_MS_rev_csv.zip',
 'household_income_20150331_MS_rev_csv.zip',
 'household_income_20150430_MS_rev_csv.zip',
 'household_income_20150531_MS_rev_csv.zip',
 'household_income_20150630_MS_rev_csv.zip',
 'household_income_20150731_MS_rev_csv.zip',
 'household_income_20150831_MS_rev_csv.zip',
 'household_income_20150930_MS_rev_csv.zip',
 'household_income_20151031_MS_rev_csv.zip',
 'househol

In [5]:
initalFile = 12
fileName = HH_Income_dir+ zipFiles[initalFile]
df = pd.read_csv(fileName, compression='zip')
if 'FAMILY_SHIFTED' not in df.columns:
    df['FAMILY_SHIFTED'] = 'N'

for f in notebook.tqdm(zipFiles[initalFile+1:72], desc='File', leave=True):
    fileName = HH_Income_dir+ f
    df_toMerge = pd.read_csv(fileName, compression='zip')
    if 'FAMILY_SHIFTED' not in df.columns:
        df_toMerge['FAMILY_SHIFTED'] = 'N'

    df = pd.concat((df,df_toMerge),axis=0)



File:   0%|          | 0/58 [00:00<?, ?it/s]

In [6]:
# df_toMerge
df

,HH_ID,STATE,HR,DISTRICT,REGION_TYPE,STRATUM,PSU_ID,MONTH_SLOT,MONTH,RESPONSE_STATUS,REASON_FOR_NON_RESPONSE,HH_WEIGHT_MS,HH_WEIGHT_FOR_COUNTRY_MS,HH_WEIGHT_FOR_STATE_MS,HH_NON_RESPONSE_MS,HH_NON_RESPONSE_FOR_COUNTRY_MS,HH_NON_RESPONSE_FOR_STATE_MS,AGE_GROUP,OCCUPATION_GROUP,EDUCATION_GROUP,GENDER_GROUP,SIZE_GROUP,TOTAL_INCOME,INCOME_OF_ALL_MEMBERS_FROM_ALL_SOURCES,INCOME_OF_ALL_MEMBERS_FROM_WAGES,INCOME_OF_ALL_MEMBERS_FROM_PENSION,INCOME_OF_ALL_MEMBERS_FROM_DIVIDEND,INCOME_OF_ALL_MEMBERS_FROM_INTEREST,INCOME_OF_HOUSEHOLD_FROM_ALL_SOURCES,INCOME_OF_HOUSEHOLD_FROM_RENT,INCOME_OF_HOUSEHOLD_FROM_SELF_PRODUCTION,INCOME_OF_HOUSEHOLD_FROM_PRIVATE_TRANSFERS,INCOME_OF_HOUSEHOLD_FROM_GOVERNMENT_TRANSFERS,INCOME_OF_HOUSEHOLD_FROM_BUSINESS_PROFIT,INCOME_OF_HOUSEHOLD_FROM_SALE_OF_ASSET,INCOME_OF_HOUSEHOLD_FROM_GAMBLING,FAMILY_SHIFTED,INCOME_OF_ALL_MEMBERS_FROM_FD_PF_INSURANCE
0,77051739,Jammu & Kashmir,HR 1,Bandipore,RURAL,HR 1_RURAL_R,PSU_ID_7959,May 2015,Jan 2015,Accepted,No Failure,2707.736909,2707.736909,2707.736909,1.103118,1.103118,1.103118,Youngsters - dominant,White-collar Clerical Employees,All Graduates household,Female Majority,8-10 Members,19000,19000,19000,0,0,0,0,0,0,0,0,0,0,0,N,NaN
1,61287374,Jammu & Kashmir,HR 1,Bandipore,RURAL,HR 1_RURAL_R,PSU_ID_7959,May 2015,Jan 2015,Accepted,No Failure,2707.736909,2707.736909,2707.736909,1.103118,1.103118,1.103118,Balanced households with no Seniors,White-collar Professional Employees,All Graduates household,Male Dominated,4 Members,20000,20000,20000,0,0,0,0,0,0,0,0,0,0,0,N,NaN
2,34049553,Jammu & Kashmir,HR 1,Bandipore,RURAL,HR 1_RURAL_R,PSU_ID_7959,May 2015,Jan 2015,Accepted,No Failure,2707.736909,2707.736909,2707.736909,1.103118,1.103118,1.103118,Grown-up - dominant,Self-employed Entrepreneurs,All Matriculates household,Male Majority,3 Members,5000,5000,5000,0,0,0,0,0,0,0,0,0,0,0,N,NaN
3,92526123,Jammu & Kashmir,HR 1,Bandipore,RURAL,HR 1_RURAL_R,PSU_ID_7959,May 2015,Jan 2015,Accepted,No Failure,2707.736909,2707.736909,2707.736909,1.103118,1.103118,1.103118,Others households of Grown-ups,Wage Labourers,Matriculates minority household,Male Majority,8-10 Members,25000,25000,25000,0,0,0,0,0,0,0,0,0,0,0,N,NaN
4,61508707,Jammu & Kashmir,HR 1,Bandipore,RURAL,HR 1_RURAL_R,PSU_ID_7959,May 2015,Jan 2015,Accepted,No Failure,2707.736909,2707.736909,2707.736909,1.103118,1.103118,1.103118,Others households of Grown-ups,Small Traders/Hawkers,Households of some literates,Balanced,4 Members,9000,9000,9000,0,0,0,0,0,0,0,0,0,0,0,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174400,18693357,Meghalaya,HR 110,East Khasi Hills,RURAL,HR 110_RURAL_R,PSU_ID_12896,Mar 2020,Nov 2019,Accepted,No Failure,780.358644,780.358644,780.358644,1.169435,1.169435,1.169435,Youngsters - dominant,Small/Marginal Farmers,Households of all literates,Male Majority,7 Members,43725,42750,42750,0,0,0,975,0,975,0,0,0,0,0,N,0.0
174401,82768393,Meghalaya,HR 110,East Khasi Hills,RURAL,HR 110_RURAL_R,PSU_ID_12896,Mar 2020,Nov 2019,Accepted,No Failure,780.358644,780.358644,780.358644,1.169435,1.169435,1.169435,Youngsters - dominant,Non-industrial Technical Employees,Households of all literates,Balanced,4 Members,18332,17632,17632,0,0,0,700,0,700,0,0,0,0,0,N,0.0
174402,18077972,Meghalaya,HR 110,East Khasi Hills,RURAL,HR 110_RURAL_R,PSU_ID_12896,Mar 2020,Nov 2019,Accepted,No Failure,780.358644,780.358644,780.358644,1.169435,1.169435,1.169435,Other households of the Young,Small/Marginal Farmers,Matriculates majority household,Balanced,4 Members,19104,18520,18520,0,0,0,584,0,584,0,0,0,0,0,N,0.0
174403,35087702,Meghalaya,HR 110,East Khasi Hills,RURAL,HR 110_RURAL_R,PSU_ID_12896,Mar 2020,Nov 2019,Accepted,No Failure,780.358644,780.358644,780.358644,1.169435,1.169435,1.169435,Other households of the Young,Small/Marginal Farmers,Households of all literates,Male Majority,8-10 Members,43000,41800,41800,0,0,0,1200,0,1200,0,0,0,0,0,N,0

In [45]:
df = df.reset_index(drop=True)
df.to_feather('proc_data\HH_Income_Panel_long.feather')
